In [1]:
import numpy as np
import pandas as pd
import matplotlib as matplot
import nltk
import sklearn as sk
import re
import scipy.sparse

# Question 1

In [2]:
en_df_raw = pd.read_csv('data/CONcreTEXT_trial_EN.tsv', sep='\t') # load data files
it_df_raw = pd.read_csv('data/CONcreTEXT_trial_IT.tsv', sep='\t')

en_df = pd.DataFrame()
it_df = pd.DataFrame()

en_df_raw.head()

,TARGET,POS,INDEX,TEXT,MEAN
0,achievement,N,3,"Bring up academic achievements , awards , and ...",3.06
1,achievement,N,9,"Please list people you have helped , your pers...",3.03
2,activate,V,1,Add activated carbon straight to your vodka .,3.83
3,activate,V,15,"Place sensors around your garden , and when a ...",5.51
4,adventure,N,9,Look for a partner that shares your level of a...,2.03


In [3]:
en_df['SENTENCES'] = en_df_raw['TEXT'].apply(lambda sent: sent.strip().lower())
it_df['SENTENCES'] = it_df_raw['TEXT'].apply(lambda sent: sent.strip().lower())

In [4]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
en_df['WORDS'] = en_df['SENTENCES'].apply(tokenizer.tokenize)
it_df['WORDS'] = it_df['SENTENCES'].apply(tokenizer.tokenize)

In [5]:
en_df['TOKENS'] = en_df['WORDS'].apply(lambda words: ["<s>"] + words + ["</s>"])
it_df['TOKENS'] = it_df['WORDS'].apply(lambda words: ["<s>"] + words + ["</s>"])
en_df['TOKENS'][0]

['<s>',
 'bring',
 'up',
 'academic',
 'achievements',
 'awards',
 'and',
 'other',
 'milestones',
 'in',
 'your',
 'life',
 '</s>']

In [6]:
def ppmi(word1, word2, sentence, W_size=3):
    occurances_word1 = occurance(word1, sentence)
    occurances_word2 = occurance(word2, sentence)
    
    cooccurances = cooccurance(word1, word2, sentence, W_size)
    
    quotient = (cooccurances / (occurances_word1 * occurances_word2) )
    return max(0, np.log2(quotient))

In [7]:
en_vocab = nltk.lm.Vocabulary([word for sentence in en_df['WORDS'] for word in sentence])
len(en_vocab)

642

In [43]:
class PPMI:
    def __init__(self, sentences):
        self.window_size = 3
        
        self.vocab = nltk.lm.Vocabulary([word for sentence in sentences for word in sentence])
        self.sentences = sentences
        self.matrix = pd.DataFrame(columns=self.vocab, index=self.vocab)
        
    def compute(self):
        count = 0
        
        for index_word in self.matrix.index:
            for column_word in self.matrix.index:
                count += 1
                
        return count
        
    def cooccurance(self, word1, word2):
        """Counts the number of co-occurances in a sentence within the window size

        word1 - first word
        word2 - second word
        sentence - sentence to use
        W_size - window size on each side, such that co-occurances will be found within the window size
        """
        count = 0
        for sentence in self.sentences:
            for index in range(0, len(sentence)):
                if sentence[index] == word1:

                    left_slice_index = max(0, index-self.window_size)
                    right_slice_index = index+self.window_size+1

                    for inner_index, inner_word in enumerate(sentence[left_slice_index : right_slice_index]):
                        if inner_word == word2: count += 1

        return count

In [44]:
a = PPMI(en_df['TOKENS'])

In [45]:
a.compute()

414736